# Neural Networks

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
print(torch.__version__)

1.0.0.dev20181017


In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0523,  0.0346,  0.1033,  0.0729, -0.0341,  0.0347, -0.1246,  0.0602,
          0.0261, -0.0219]], grad_fn=<ThAddmmBackward>)


## nn.Module.zero_grad

model의 parameter들을 모두 zero로 만든다.  

<br/>

```python
def zero_grad(self):
        r"""Sets gradients of all model parameters to zero."""
        for p in self.parameters():
            if p.grad is not None:
                p.grad.detach_()
                p.grad.zero_()
```

<br/>

### detach_()
     
Tensor를 만든 graph에서 분리해서 leaf로 만든다.  
`views`는 in-pace로 detach할 수 없다.
> Detaches the Tensor from the graph that created it, making it a leaf. Views cannot be detached in-place.
    
    
### zero_
???  

결국, `zero_grad`는 model의 모든 parameter들의 gradient를 zero로 만들어주는 것.  

> 왜 `detach_`후에 `zero_`를 하는가?


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Loss function

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.3100, grad_fn=<MseLossBackward>)


In [24]:
func1 = loss.grad_fn.next_functions
func2 = func1[0][0].next_functions
func3 = func2[0][0].next_functions
func4 = func3[0][0].next_functions

print(loss.grad_fn)
print(func1)
print(func2)
print(func3)
print(func4)

((<ThAddmmBackward object at 0x7fb5b2334eb8>, 0),)
((<ExpandBackward object at 0x7fb5a82e0c18>, 0), (<ReluBackward object at 0x7fb5a82e0c88>, 0), (<TBackward object at 0x7fb5a82e0cf8>, 0))
((<AccumulateGrad object at 0x7fb5a82e0cc0>, 0),)
()


### loss.grad_fn.next_functions ??
    
**Iterable**인가? 어떻게 구성되어있는 놈이지?  
단순히 `next_functions`로만 구성이 되어있다.  
이는 그냥 단방향 linked list라고 생각하면 될까?

In [25]:
from collections.abc import Iterable
print(isinstance(loss.grad_fn, Iterable))

print(dir(loss.grad_fn))

False
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_register_hook_dict', 'metadata', 'name', 'next_functions', 'register_hook', 'requires_grad']


## Backprop
     
Backpropagation은 `loss.backward()`로 구할 수 있는데,  
Backpropagation을 구할 때는 항상 존재하던 gradient를 삭제해야한다.  
만약 그렇지 않다면, 기존의 gradient가 누적될 것이다.
> 이는 momory를 추가로 사용한다는 의미와 같고,  
> Iteration이 지속되면, OOM; Out of Memory를 유발한다.

In [26]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0229, -0.0012, -0.0085, -0.0097, -0.0071,  0.0206])


## Update the weights

<br/>

`sub_` operation은 in-pace operation,  
`data`라는 field는 weights 혹은 gradient를 의미하는 듯 하다.

In [27]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

### How to use various different update rules? USING `torch.optim` PACKAGE!

In [28]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update